<a href="https://colab.research.google.com/github/fire-long/4J/blob/model3/%EB%AA%A8%EB%8D%B8%EB%A7%813_%EB%B0%95%EC%A7%84%EC%98%81_%EC%BD%94%EB%93%9C%EC%88%98%EC%A0%95%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

기존 코드는 GPU에 너무 큰 부담을 줌
그래서 앙상블 대신 monologg/kobert 모델만 사용하되, 이전에 했떤 코드를 필사하면서 진행

In [ ]:
#torch_optimizer : 파이토치 기반. 딥러닝 모델 최적화를 위한 여러 알고리즘 제공
!pip install torch_optimizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 315.6 kB/s eta 0:00:00


In [ ]:
#transformers : 딥러닝 NLP 모델 구현 라이브러리
#사전 훈련(pre-trained) 언어 모델(LM)을 제공하고, 이후에 이걸 특정 자연어 처리 업무에 맞게 fine-tunning을 하는 것임
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.0 MB/s eta 0:00:00


- pre-trained
  - 사전 학습 모델 : 대규모 데이터셋에서 미리 학습된 모델
    - 다른 자연어 처리 task에서도 유용하게 됨.
  - 학습시간을 줄이고 소규모 데이터셋에서도 고성능 결과 얻을 수 있음
- fine-tunning
  - 사전 훈련 모델을 사용해 특정 자연어 처리 업무에 맞는 모델을 만드는 과정

# 시작

In [ ]:
#importing libraries
import numpy as np
import os
import pickle
import sys
import pandas as pd
import re
import cv2

- pickle : 파이썬 객체를 직렬화/역직렬화 하기 위한 모듈. 파이썬 객체를 디스크에 저장하고 로드할 수 있는 기능
- sys : '인터프리터가 쓰거나 유지 및 관리하는 일부변수'와 인터프리터 간 상호작용 함수를 제공
- OpenCV(cv2) : 컴퓨터 비전 및 이미지 처리를 위한 라이브러리

In [ ]:
import torch #인공신경망 네트워크를 설계할 때 쓰는 오픈 소스 머신러닝 프레임워크, 파이토치.
#tensor 연산자, 인공신경망 모델, 여러 최적화 알고리즘을 제공함
import torch.cuda.amp as amp
#Automatic Mixed Precision
#: 단정밀도 및 반정밀도 부동 소수점 연산의 조합을 씀 -> GPU 메모리 사용량을 줄이고 학습 속도를 높임

In [ ]:
import torch.nn as nn
#파이토치의 Neural Network 모듈

In [ ]:
from torch.optim.optimizer import Optimizer, required
#Optimizer : 최적화 알고리즘을 파이토치에서 구현 및 사용하고자 제공되는 공통 인터페이스를 가져옴
#required : 함수에 필요한 인수를 지정할 때 사용되는 필수 함수를 가져옴
import torch_optimizer as optim
#torch_optimizer : 파이토치용 서드 파티 라이브러리. 추가 최적화 알고리즘을 제공함.

In [ ]:
#데이터 로딩을 위한 라이브러리들#


from torch.utils.data import DataLoader
#DataLoader : 파이토치 데이터셋에 대한 반복자를 제공
# -> 데이터셋에서 한 번에 하나의 샘플을 가져와 처리하기보다 배치 단위로 처리하는 게 효율적이라서
#    데이터셋의 모든 샘플을 각 반복자가 데이터셋의 한 배치를 순차적으로 반환해줌.

from torch.utils.data.dataset import Dataset
#Dataset : 파이토치에서 데이터셋을 나타내는 추상 클래스
#데이터 샘플에 대한 액세스를 제공하기 위해 어떤 하위 클래스든지 __len__ 및 __getitem__ 두 가지 메서드를 구현해야 함
##그래서 이후 model 코드 작성할 때 두 메서드 구현하는 내용이 나옴.

from torch.utils.data.sampler import *
#Sampler: 데이터셋에서 요소를 샘플링하는 전략을 나타내는 추상 클래스
#         DataLoader 클래스와 함께 사용하여 데이터 로드 중 샘플링 동작을 사용자 정의할 수 있음
'''
파이토치는 RandomSampler(요소를 무작위로 샘플링),
SequentialSampler(요소를 순차적으로 샘플링) 및 SubsetRandomSampler(요소의 일부를 무작위로 샘플링)
과 같은 Sampler 클래스의 여러 구현을 제공함'''

'\n파이토치는 RandomSampler(요소를 무작위로 샘플링), \nSequentialSampler(요소를 순차적으로 샘플링) 및 SubsetRandomSampler(요소의 일부를 무작위로 샘플링)\n과 같은 Sampler 클래스의 여러 구현을 제공함'

In [ ]:
import math
#수학적인 함수 및 상수를 제공

from collections import defaultdict
#collections.defaultdict : 기본값을 갖는 딕셔너리.
#딕셔너리에 존재하지 않는 키를 조회하면 딕셔너리에 있는 기본값을 반환해줌으로써
#KeyError를 방지하고 코드를 더 간결하게 만듦

import itertools as it
#반복자(iterator)를 생성하고 연산하는 함수들의 모음
#반복적인 작업을 진행할 때 메모리 사용량을 최소화할 수 있음

import tqdm
#진행 바 같은 걸 시각화해줌

In [ ]:
import random #난수 생성, 무작위 샘플링에서 사용됨
import matplotlib.pyplot as plt #시각화를 위함
from timeit import default_timer as timer
#코드 실행 시간을 측정하는 함수 제공해서, 알고리즘 효율성을 파악할 수 있음

In [ ]:
from sklearn.model_selection import KFold #kfold
from sklearn.metrics import f1_score #f1-score
from sklearn.preprocessing import LabelEncoder #라벨인코딩.
from sklearn.preprocessing import StandardScaler #표준화

from sklearn.metrics import accuracy_score #정확도 측정

In [ ]:
#transformer 모델들과 tokenizer를 import
##모델들은 다 사전학습되어있음.
from transformers import XLMPreTrainedModel, XLMRobertaModel, XLMRobertaConfig, XLMRobertaTokenizer
#XLMPreTrainedModel : XLM 모델을 기반으로 하는 모든 Pretrained Model의 베이스 클래스


from transformers import XLMRobertaForSequenceClassification, BertForSequenceClassification
from transformers import AutoTokenizer
from transformers import BertForSequenceClassification, DistilBertForSequenceClassification, XLNetForSequenceClassification,\
XLMRobertaForSequenceClassification, XLMForSequenceClassification, RobertaForSequenceClassification

In [ ]:
#AdamW 최적화 알고리즘을 사용
from transformers import AdamW
#학습률 조절에 사용하는 함수
from transformers import get_linear_schedule_with_warmup

In [ ]:
# class args
class args:  #여러 학습 인자의 값을 설정
    # ---- factor ---- #
    debug=False #디버그 모드 여부
    amp = True #mixed precision을 사용할 지 여부
    gpu = '0' #GPU 번호

    epochs=10 #에포크 수
    batch_size=1 #배치 크기
    weight_decay=1e-6 #가중치 감소 값
    n_fold=5 #교차 검증(fold) 수
    fold=3 # [0, 1, 2, 3, 4] # 원래는 3

    exp_name = 'experiment_name_folder'
    dir_ = f'./saved_models/'
    pt = 'your_model_name'
    max_len = 33 #모델 입력 시퀀스의 최대 길이 설정
    #학습률을 설정하는 파라미터
    start_lr = 1e-5#1e-3,5e-5 #학습시 초기학습률 값을 의미. 학습 초기에 빠르게 수렴할 가능성이 있지만, 값이 너무 크면 수렴이 불안정해짐
    min_lr=1e-6 #학습 중 학습율이 더이상 작아지지 않게 제한하는 값. 해당 값 이하로 떨어지지 않게 보장함.
    # ---- Dataset ---- #

    # ---- Else ---- #
    num_workers=8
    seed=2021
    scheduler = None#모델의 학습률을 조정하는 방법을 지정. 방법은 사용하지 않겠지만, 다른 변수를 이용해 학습률 조정 방식을 선택할 수 있음.


data_dir = './' # 데이터가 저장된 디렉토리 경로를 나타냄
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu #args.gpu : 코드 실행 시 지정된 GPU의 인덱스
device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu") #디바이스 유형 설정.
#만약 CUDA 디바이스가 사용 가능하면 "cuda"로 설정하고, 그렇지 않으면 "cpu"로 설정함. 이를 통해 디바이스 유형에 맞게 모델을 초기화할 수 있음.

##모델 학습 시 랜덤 시드를 설정해 재현성을 보장
def set_seeds(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    #torch.backends.cudnn : NVIDIA cuDNN 라이브러리를 사용해서 PyTorch 연산의 실행 속도를 높이는 데 사용됨.
    #cuDNN : Deep Neural Network 라이브러리, NVIDIA GPU에서 딥러닝 모델을 학습 및 추론할 때 연산속도를 높이기 위한 최적화된 기능을 제공
    #torch.backends.cudnn을 쓰면 이런 최적화를 활용해 PyTorch 연산 속도를 높일 수 있음.
    torch.backends.cudnn.deterministic = True #기본값이 True. 연산 실행 시간을 단축하고자 cuDNN 라이브러리가 실행 시간을 측정하고 최적화를 수행하는 걸 의미
    torch.backends.cudnn.benchmark = False # 연산 결과가 항상 동일하지 않을 수 있다는 걸 의미함. -> 연산 속도 및 학습을 빠르게함

set_seeds(seed=args.seed)


In [ ]:
# - util - #
def get_learning_rate(optimizer): #현재 학습률을 가져와서 반환하는 함수
    lr=[]
    for param_group in optimizer.param_groups:
      #param_groups : optimizer의 학습률과 weight decay 같은 하이퍼파라미터 관리
        lr +=[ param_group['lr'] ]

    assert(len(lr)==1)
    #'lr'의 길이가 1인지 확인하는데, 이건 현재 이 코드에서 하나의 학습률만 사용하기 때문임.
    lr = lr[0]

    return lr

def load_data(): #데이터 로드 합수
    train=pd.read_csv('./train_data.csv')
    test=pd.read_csv('./test_data.csv')

    #일부 column을 지정
    train=train[['title','topic_idx']]
    test=test[['title']]
    #5-fold 교차 검증 수행
    from sklearn.model_selection import StratifiedKFold
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    train['fold'] = -1
    for n_fold, (_,v_idx) in enumerate(skf.split(train, train['topic_idx'])):
        train.loc[v_idx, 'fold']  = n_fold
    #train 데이터에 fold와 id열을 추가
    train['id'] = [x for x in range(len(train))]

    return train, test

# 전처리

In [ ]:
# make KoBertTokenizer
# 한글을 토큰화할 수 있는 KoBert 모델용 custom tokenizer를 만드는 데 사용함.
import logging
import os
import unicodedata #유니코드 문자 처리를 위함
from shutil import copyfile #파일 복사

from transformers import PreTrainedTokenizer

logger = logging.getLogger(__name__)
#logger : 스크립트 실행되는 동안 메세지를 기록하는 데 사용함.
# 로거명을 __name__로 설정하면 현재 모듈명을 로거의 이름으로 사용함
# 이렇게 하면 각 모듈이 자체 로거를 가질 수 있으므로 대규모 애플리케이션에서 로그 메세지를 더 잘 구성할 수 있음.

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model", # 어휘파일의 위치 지정
                     "vocab_txt": "vocab.txt"} # 어휘에 대한 토큰-index mapping 사전을 구축하는 데 쓰임.
#PRETRAINED_VOCAB_FILES_MAP 변수를 정의하는 코드
# hugging face의 transformer 라이브러리에서 사용되고, 사전 학습된 모델의 tokenizer와 관련된 파일들의 URL을 매핑함
#즉, 해당 모델에 대한 토크나이저와 어휘파일을 쉽게 다운로드할 수 있음.
PRETRAINED_VOCAB_FILES_MAP = {
    #vocab_file과 vocab_txt는 각각 토크나이저 파이과 어휘 파일의 URL을 가리키는 사전임
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        #각각의 어휘파일의 링크가 제공됨.
        #이 링크들은 Hugging Face에서 제공하는 S3 버킷에서 해당 어휘 파일을 다운로드할 수 있는 주소를 제공함.
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    #세 개의 다른 모델명, 각각 해당 모델에 대한 토크나이저와 어휘 파일을 가짐.
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

#KoBERT의 tokenizer와 관련된 변수들을 정의하는 부분
#positional embedding의 크기를 지정
PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}
#초기화 설정값 지정.
PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False}, #do_lower_case를 False로 설정하면 대소문자를 구분함.
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'
#KOBERT tokenizer에서 사용되는 특수 토큰.
#"__"은 subword를 나타내며, 이는 단어 일부분이 다른 단어와 함께 subword 단위로 나눠지는 경우 사용됨.

class KoBertTokenizer(PreTrainedTokenizer):
  # KoBertTokenizer 클래스 : 문장 조각 기반 tokenizer를 구축하는 데 사용하는 transformer library의 PreTrainedTokenizer class를 상속받음
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
          #vocabtxt 파일을 열어서 어휘(vocabulary) 만듦.
          #해당 텍스트 파일은 각 단어가 한 줄씩 적혀 있음
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        #self.max_len_single_sentence = self.max_len - 2  # take into account special tokens
        #self.max_len_sentences_pair = self.max_len - 3  # take into account special tokens

        #sentencepiece 패키지를 이용해, SentencePiece모델 파일(tokenizer_78b3253a26.model)을 로드함.
        #이 모델 파일은 BytePairEncoding 기반의 subword tokenization 정보를 담고 있으며,
        #각 subwork token을 생서하기 위한 기준 정보를 담음
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        #KoBertTokenizer 객체의 어휘 변수로 어휘 파일(vocab_txt), SentencePiece 모델 파일(vocab_file) 경로,
        #대소문자 구분 여부(do_lower_case), 공백 제거 여부(remove_space), 악센트 유지 여부(keep_accents) 등이 저장됨
        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor() #sentencepiece 패키지를 사용하여 SentencePieceProcessor 객체를 초기화
        self.sp_model.Load(vocab_file)

    @property
    #메서드를 클래스 속성의 "getter"로 정의할 수 있는 파이썬 데코레이터.
    #메서드 저의 앞에서 @property를 쓰면 해당 메서드를 클래스 인스턴스의 속성으로 접근 가능함.
    #(ex. KoBertTokenizer 클래스에서 vocab_size는 속성이지만,
    #KoBertTokenizer 인스턴스의 속성처럼 전근 가능하되 실제 토큰화 어휘의 길이를 반환하는 method)
    def vocab_size(self):
        return len(self.idx2token)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ 어휘를 사용하여 아이디의 토큰(문자열/유니코드)을 변환함 """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """어휘를 사용하여 인덱스(정수)를 토큰(문자열/유니코드)으로 변환함"""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """토큰 시퀀스(하위 단어의 문자열)를 단일 문자열로 변환함"""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        특수 토큰을 연결하고 추가하여 시퀀스 또는 시퀀스 쌍에서 시퀀스 분류 작업을 위한 모델 입력을 구축함.
        RoBERTa 시퀀스의 형식은 다음과 같다:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        특수 토큰이 추가되지 않은 토큰 목록에서 시퀀스 ID를 검색함.
        이 메서드는 토큰화기 ``prepare_for_model`` 또는 ``encode_plus`` 메서드를 사용하여 특수 토큰을 추가할 때 호출됨.
        Args:
            token_ids_0: ID 목록(특수 토큰을 포함하지 않아야 함)
            token_ids_1: 시퀀스 쌍에 대한 시퀀스 ID를 가져올 때 필요한 선택적 ID 목록(특수 토큰을 포함하지 않아야 함)
            already_has_special_tokens: (default False) 토큰 목록이 모델에 대한 특수 토큰으로 이미 형식화되어 있으면 True로 설정
        Returns:
            [0, 1] 범위의 정수 목록: 특수 토큰은 0, 시퀀스 토큰은 1
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        전달된 두 시퀀스로부터 시퀀스 쌍 분류 작업에 사용할 마스크를 만듦.
        BERT 시퀀스 쌍 마스크의 형식은 다음과 같음:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        token_ids_1이 None이면 마스크의 첫 번째 부분(0)만 반환함
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ 문장 조각 어휘(원본 파일 복사)와 특수 토큰 파일을 디렉토리에 저장"""
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "{}에 어휘 저장:어휘 인덱스가 연속적이지 않음."
                        " 어휘가 손상되지 않았는지 확인하세요!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

In [ ]:
#bert_tokenizer는 preprocessing_train() 및 preprocessing_test() 함수에서 각 문장을 토큰화 및 인코딩하는 데 사용됨.
#이 함수는 문장, MAX_LEN 매개변수, 토큰화 객체를 입력으로 받아 input_ids, attention_mask, token_type_ids 텐서를 반환함
def bert_tokenizer(sent, MAX_LEN, tokenizer):

    encoded_dict=tokenizer.encode_plus(
    text = sent,
    add_special_tokens=True,
    max_length=MAX_LEN,
    pad_to_max_length=True,
    return_attention_mask=True,
    truncation = True)

    input_id=encoded_dict['input_ids']
    attention_mask=encoded_dict['attention_mask']
    #token_type_id = encoded_dict['token_type_ids']
    token_type_id = 0

    return input_id, attention_mask, token_type_id
#텍스트 분류 작업을 위해서, 각각 훈련데이터와 테스트 데이터의 전처리 단계를 수행하는
#preprocessing_train() 및 preprocessing_test()의 두 가지 함수를 정의

'''
이 함수들은 BERT 기반 토큰화기를 사용하여 입력 텍스트를 토큰화하고
토큰을 BERT 기반 모델을 미세 조정하는 데 필요한 입력인 input_ids, attention_masks, token_type_ids로 인코딩함

인코딩된 입력은 훈련 세트의 경우 train_data, 테스트 세트의 경우 test_data라는 딕셔너리에 저장된 다음 피클 파일 형식으로 디스크에 저장됨
파일 저장 경로는 사전 학습된 모델 이름 또는 디렉터리인 args.pt 매개변수를 기반으로 함.
'''
def preprocessing_train():

    pt = args.pt#'monologg/kobert'

    if 'kobert' in pt:
        tokenizer = KoBertTokenizer.from_pretrained(pt,  cache_dir='bert_ckpt', do_lower_case=False) #do_lower_case : 모든 문자를 소문자로 변환할지 여부설정
        print('load kobert')
    else:
        tokenizer = AutoTokenizer.from_pretrained(args.pt)

    MAX_LEN = args.max_len
    #MAX_LEN 매개변수는 입력 시퀀스의 최대 길이를 지정하며, 토큰화기는 이 길이에 맞게 시퀀스를 자르거나 패딩
    train = pd.read_csv('./train_data.csv')
    train=train[['title','topic_idx']]

    input_ids =[]
    attention_masks =[]
    token_type_ids =[]
    train_data_labels = []

    for train_sent, train_label in tqdm.tqdm(zip(train['title'], train['topic_idx'])):
        try:
            input_id, attention_mask,_ = bert_tokenizer(train_sent, MAX_LEN=MAX_LEN, tokenizer=tokenizer)

            input_ids.append(input_id)
            attention_masks.append(attention_mask)
            token_type_ids.append(0)
            #########################################
            train_data_labels.append(train_label)

        except Exception as e:
            print(e)
            pass

    train_input_ids=np.array(input_ids, dtype=int)
    train_attention_masks=np.array(attention_masks, dtype=int)
    train_token_type_ids=np.array(token_type_ids, dtype=int)
    ###########################################################
    train_inputs=(train_input_ids, train_attention_masks, train_token_type_ids)
    train_labels=np.asarray(train_data_labels, dtype=np.int32)

    # save
    train_data = {}

    train_data['input_ids'] = train_input_ids
    train_data['attention_mask'] = train_attention_masks
    train_data['token_type_ids'] = train_token_type_ids
    train_data['targets'] = np.asarray(train_data_labels, dtype=np.int32)

    os.makedirs(f'./data/{pt}/', exist_ok=True)
    with open(f'./data/{pt}/train_data_{MAX_LEN}.pickle', 'wb') as f:
        pickle.dump(train_data, f, pickle.HIGHEST_PROTOCOL)

def preprocessing_test():

    pt = args.pt
    if 'kobert' in pt:
        tokenizer = KoBertTokenizer.from_pretrained(pt,  cache_dir='bert_ckpt', do_lower_case=False)
        print('load kobert')
    else:
        tokenizer = AutoTokenizer.from_pretrained(args.pt)
    MAX_LEN = args.max_len

    test = pd.read_csv('./test_data.csv')
    test=test[['title']]

    input_ids =[]
    attention_masks =[]
    token_type_ids =[]

    for test_sent in tqdm.tqdm(test['title']):
        try:
            input_id, attention_mask,_ = bert_tokenizer(test_sent, MAX_LEN=MAX_LEN, tokenizer=tokenizer)

            input_ids.append(input_id)
            attention_masks.append(attention_mask)
            token_type_ids.append(0)
            #########################################

        except Exception as e:
            print(e)
            pass

    test_input_ids=np.array(input_ids, dtype=int)
    test_attention_masks=np.array(attention_masks, dtype=int)
    test_token_type_ids=np.array(token_type_ids, dtype=int)
    ###########################################################
    test_inputs=(test_input_ids, test_attention_masks, test_token_type_ids)


    # save
    test_data = {}

    test_data['input_ids'] = test_input_ids
    test_data['attention_mask'] = test_attention_masks
    test_data['token_type_ids'] = test_token_type_ids

    os.makedirs(f'./data/{pt}/', exist_ok=True)
    with open(f'./data/{pt}/test_data_{MAX_LEN}.pickle', 'wb') as f:
        pickle.dump(test_data, f, pickle.HIGHEST_PROTOCOL)


In [ ]:
!pip install sentencepiece # spm은 KoBertTokenizer class의 일부이고, args.pt 변수 값에 따라 다른 토크나이저가 선택됨.
#오류 메세지가 떴을 때는 KoBertTokenizer 대신에 BertTokenizer가 선택되어 있는데, 이는 KoBertTokenizer가 의존하는 SentencePiece 패키지가 안 설치 되어 있기 때문임

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/JJJJ

/content/drive/MyDrive/JJJJ


In [ ]:
##이름과 입력값을 보면 주어진 사전 학습된 모델에 사용하기 위해 훈련 및 테스트 데이터에 대해 일종의 전처리를 수행##
for pt, max_len in zip(['monologg/kobert','klue/roberta-base','klue/roberta-small','klue/roberta-large','xlm-roberta-large',
           'bert-base-multilingual-uncased', 'klue/roberta-large'],[33,33,33,33,33,33,28]):
  #각 반복마다 args.max_len 및 args.pt 변수를 각각 max_len 및 pt 값으로 설정
    args.max_len = max_len
    args.pt = pt
    #그런 다음 업데이트된 args 변수를 사용하여 preprocessing_train() 및 preprocessing_test() 함수를 호출
    preprocessing_train()
    preprocessing_test()
    #마지막으로 현재 모델에 대한 전처리가 완료되었음을 나타내는 메시지를 인쇄
    print(f'{args.pt} 모델 전처리 완료')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


load kobert


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
45654it [00:15, 2856.12it/s]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


load kobert


  0%|          | 0/9131 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 9131/9131 [00:02<00:00, 3044.79it/s]


...

xlm-roberta-large 모델 전처리 완료


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
45654it [00:07, 6278.05it/s]
  0%|          | 0/9131 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_

bert-base-multilingual-uncased 모델 전처리 완료


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
45654it [00:08, 5699.07it/s] 
  0%|          | 0/9131 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max

klue/roberta-large 모델 전처리 완료


# 모델

In [ ]:
class KobertDataSet(Dataset):#파이토치를 이용한 데이터셋 클래스

    def __init__(self, data, test=False): #데이터와 테스트 여부(test)를 받아서 저장

        self.data = data
        self.test = test

    def __len__(self): #데이터 길이 반환. 여기선 input_ids의 길이를 기준으로 함.

        return self.data['input_ids'].shape[0]

    def __getitem__(self,idx): #인덱스(idx)를 받아 해당 인덱스의 데이터 반환
        #이 데이터는 input_ids, attention_mask, token_type_ids 그리고 targets로 이뤄짐
        ids = torch.tensor(self.data['input_ids'][idx], dtype=torch.long)
        mask = torch.tensor(self.data['attention_mask'][idx], dtype=torch.long)
        token_type_ids = torch.tensor(self.data['token_type_ids'][idx], dtype=torch.long)
        #torch.long : long(정수형, 64bits)로 형변환

        if self.test: #test 데이터인 경우에는 targets가 없으므로 ids, mask, token_type_ids만 반환
            return {
                'ids': ids,
                'mask': mask,
                'token_type_ids': token_type_ids
            }

        else:
            target = torch.tensor(self.data['targets'][idx],dtype=torch.long) #train 데이터는 targets도 반환
            return {
                    'ids': ids,
                    'mask': mask,
                    'token_type_ids': token_type_ids,
                    'targets': target
                }

# k-fold 안 하고 monologg/kobert만 사용해보기

In [ ]:
# ------------------------
#  scheduler
# ------------------------
#do_valid와 do_predict 함수는 모델 학습 파이프라인에서 각각 모델의 유효성을 검사하고 예측하는 데 사용
def do_valid(net, valid_loader):
#학습된 모델과 유효성 검사 데이터로더를 받아서 모델 유효성 검사 손실, F1 score 및 정확도를 반환함.
#예측된 출력확률, 대상레이블, 유효성 검사 데이터의 로그도 반환함.
    val_loss = 0
    target_lst = []
    pred_lst = []
    logit = []
    loss_fn = nn.CrossEntropyLoss()

    net.eval()
    start_timer = timer()
    for t, data in enumerate(tqdm.tqdm(valid_loader)):
        ids  = data['ids'].to(device)
        mask  = data['mask'].to(device)
        tokentype = data['token_type_ids'].to(device)
        target = data['targets'].to(device)

        with torch.no_grad():
            if args.amp:
                with amp.autocast():
                    # output
                    output = net(ids, mask)
                    output = output[0]

                    # loss
                    loss = loss_fn(output, target)

            else:
                output = net(ids, mask)#.squeeze(0)
                loss = loss_fn(output, target)

            val_loss += loss
            target_lst.extend(target.detach().cpu().numpy())
            pred_lst.extend(output.argmax(dim=1).tolist())
            logit.extend(output.tolist())

        val_mean_loss = val_loss / len(valid_loader)
        validation_score = f1_score(y_true=target_lst, y_pred=pred_lst, average='macro')
        validation_acc = accuracy_score(y_true=target_lst, y_pred=pred_lst)


    return val_mean_loss, validation_score, validation_acc, logit

def do_predict(net, valid_loader):
#학습된 모델과 테스트데이터로더를 받아 테스트 데이터의 예측된 레이블과 로그를 반환함

    val_loss = 0
    pred_lst = []
    logit=[]
    net.eval()
    for t, data in enumerate(tqdm.tqdm(valid_loader)):
        ids  = data['ids'].to(device)
        mask  = data['mask'].to(device)
        tokentype = data['token_type_ids'].to(device)

        with torch.no_grad():
            if args.amp:
                with amp.autocast():
                    # output
                    output = net(ids, mask)[0]

            else:
                output = net(ids, mask)

            pred_lst.extend(output.argmax(dim=1).tolist())
            logit.extend(output.tolist())

    return pred_lst,logit
"""
두 함수 모두 손실함수로, nn.CorssEntropyLoss()를 사용해 예측된 레이블과 대상 레이블 사이의 손실을 계산함.
모델을 평가 모드로 전환하려면 net.eval()을 사용해서 드롭아웃을 비활성화하고, 일괄 정규화를 평가 모드로 설정함.
유효성 검사 및 테스트 중에 기울기를 추적하지 않으려면 torch.no_grad()를 사용
"""

def run_train(folds=3):
    out_dir = args.dir_+ f'/fold{args.fold}/{args.exp_name}/'
    os.makedirs(out_dir, exist_ok=True)

    # load dataset
    train, test = load_data()
    with open(f'./data/{args.pt}/train_data_{args.max_len}.pickle', 'rb') as f:
        train_data = pickle.load(f)
    with open(f'./data/{args.pt}/test_data_{args.max_len}.pickle', 'rb') as f:
        test_data = pickle.load(f)

    # split fold
    for n_fold in range(5):
        if n_fold != folds:
            print(f'{n_fold} fold pass'+'\n')
            continue

        if args.debug:
            train = train.sample(1000).copy()

        trn_idx = train[train['fold']!=n_fold]['id'].values
        val_idx = train[train['fold']==n_fold]['id'].values


        train_dict = {'input_ids' : train_data['input_ids'][trn_idx] , 'attention_mask' : train_data['attention_mask'][trn_idx] ,
                      'token_type_ids' : train_data['token_type_ids'][trn_idx], 'targets' : train_data['targets'][trn_idx]}
        val_dict = {'input_ids' : train_data['input_ids'][val_idx] , 'attention_mask' : train_data['attention_mask'][val_idx] ,
                      'token_type_ids' : train_data['token_type_ids'][val_idx], 'targets' : train_data['targets'][val_idx]}

        ## dataset ------------------------------------
        train_dataset = KobertDataSet(data = train_dict)
        valid_dataset = KobertDataSet(data = val_dict)
        trainloader = DataLoader(dataset=train_dataset, batch_size=args.batch_size,
                                 num_workers=8, shuffle=True, pin_memory=True)
        validloader = DataLoader(dataset=valid_dataset, batch_size=args.batch_size,
                                 num_workers=8, shuffle=False, pin_memory=True)

        ## net ----------------------------------------
        scaler = amp.GradScaler()
        if 'xlm-roberta' in args.pt:
            net = XLMRobertaForSequenceClassification.from_pretrained(args.pt, num_labels = 7)

        elif 'klue/roberta' in args.pt:
            net = RobertaForSequenceClassification.from_pretrained(args.pt, num_labels = 7)
        else:
            net = BertForSequenceClassification.from_pretrained(args.pt, num_labels = 7)

        net.to(device)
        if len(args.gpu)>1:
            net = nn.DataParallel(net)

        # ------------------------
        # loss
        # ------------------------
        loss_fn = nn.CrossEntropyLoss()

        # ------------------------
        #  Optimizer
        # ------------------------
        optimizer = optim.Lookahead(optim.RAdam(filter(lambda p: p.requires_grad,net.parameters()), lr=args.start_lr), alpha=0.5, k=5)

        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = len(trainloader)*args.epochs)


        # ----
        start_timer = timer()
        best_score = 0

        for epoch in range(1, args.epochs+1):
            train_loss = 0
            valid_loss = 0

            target_lst = []
            pred_lst = []
            lr = get_learning_rate(optimizer)
            print(f'-------------------')
            print(f'{epoch}epoch start')
            print(f'-------------------'+'\n')
            print(f'learning rate : {lr : .6f}')
            for t, data in enumerate(tqdm.tqdm(trainloader)):

                # one iteration update  -------------
                ids  = data['ids'].to(device)
                mask  = data['mask'].to(device)
                tokentype = data['token_type_ids'].to(device)
                target = data['targets'].to(device)

                # ------------
                net.train()
                optimizer.zero_grad()


                if args.amp:
                    with amp.autocast():
                        # output
                        output = net(ids, mask)
                        output = output[0]

                        # loss
                        loss = loss_fn(output, target)
                        train_loss += loss


                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()

                else:
                    # output
                    output = net(ids, mask)

                    # loss
                    loss = loss_fn(output, target)
                    train_loss += loss

                    # update
                    loss.backward()
                    optimizer.step()


                # for calculate f1 score
                target_lst.extend(target.detach().cpu().numpy())
                pred_lst.extend(output.argmax(dim=1).tolist())


                if scheduler is not None:
                    scheduler.step()
            train_loss = train_loss / len(trainloader)
            train_score = f1_score(y_true=target_lst, y_pred=pred_lst, average='macro')
            train_acc = accuracy_score(y_true=target_lst, y_pred=pred_lst)

            # validation
            valid_loss, valid_score, valid_acc, _ = do_valid(net, validloader)


            if valid_acc > best_score:
                best_score = valid_acc
                best_epoch = epoch
                best_loss = valid_loss

                torch.save(net.state_dict(), out_dir + f'/{folds}f_{epoch}e_{best_score:.4f}_s.pth')
                print('best model saved'+'\n')


            print(f'train loss : {train_loss:.4f}, train f1 score : {train_score : .4f}, train acc : {train_acc : .4f}'+'\n')
            print(f'valid loss : {valid_loss:.4f}, valid f1 score : {valid_score : .4f}, valid acc : {valid_acc : .4f}'+'\n')


        print(f'best valid loss : {best_loss : .4f}'+'\n')
        print(f'best epoch : {best_epoch }'+'\n')
        print(f'best accuracy : {best_score : .4f}'+'\n')

def run_predict(model_path):
    ## dataset ------------------------------------
    # load
    with open(f'./data/{args.pt}/test_data_{args.max_len}.pickle', 'rb') as f:
        test_dict = pickle.load(f)

    print('test load')
    test_dataset = KobertDataSet(data = test_dict, test=True)
    testloader = DataLoader(dataset=test_dataset, batch_size=args.batch_size,
                             num_workers=8, shuffle=False, pin_memory=True)
    print('set testloader')
    ## net ----------------------------------------
    scaler = amp.GradScaler()
    if 'xlm-roberta' in args.pt:
        net = XLMRobertaForSequenceClassification.from_pretrained(args.pt, num_labels = 7)

    elif 'klue/roberta' in args.pt:
        net = RobertaForSequenceClassification.from_pretrained(args.pt, num_labels = 7)
    else:
        net = BertForSequenceClassification.from_pretrained(args.pt, num_labels = 7)

    net.to(device)

    if len(args.gpu)>1:
        net = nn.DataParallel(net)

    f = torch.load(model_path)
    net.load_state_dict(f, strict=True)  # True
    print('load saved models')
    # ------------------------
    # validation
    preds, logit = do_predict(net, testloader) #outputs

    print('complete predict')

    return preds, np.array(logit)
# 예측된 레이블과 로그는 각각 pred_lst 및 logit 목록에 저장


In [ ]:
"""5fold 전용"""
if __name__ == '__main__':
  #스크립트를 모듈로 가져오는 것이 아니라 직접 실행할 때만 실행해야 하는 코드를 지정할 수 있는 Python 코드의 일반적인 패턴

    for pt, max_len in zip(['monologg/kobert'],[33]):
        #일부 매개변수(args.max_len, args.pt, args.exp_name)를 정의한 다음
        args.max_len = max_len
        args.pt = pt
        args.exp_name = str(args.pt) + '_' + str(args.max_len)
        #교차 유효성 검사의 5배수에 걸쳐 pt와 max_len의 각 조합에 대해 run_train 함수를 호출
        #for i in [0,1,2,3,4]: # 5fold
        for i in [0]: #1fold
            run_train(folds=i)
  #5 교차 유효성 검사를 사용해 다양한 최대 시퀀스 길이(max_len)을 가진 사전 학습 언어 모델(pt)에 학습 스크립트를 실행하는 루프

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-------------------
1epoch start
-------------------

learning rate :  0.000010


  0%|          | 0/36523 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
 38%|███▊      | 13737/36523 [1:11:41<1761:44:56, 278.34s/it]

In [ ]:
#여러 transformer 모델을 앙상블시킴
def ensemble():
    final_logit=0 #final_logit 변수를 0으로 초기화한 다음 각 모델에서 예측 로그를 추가함.
    #각 모델은 예측된 레이블과 예측 로그를 반환하는 run_predict() 함수를 사용해 호출됨
    args.max_len=33
    args.pt = 'monologg/kobert'
    _, logit1 = run_predict("./saved_models/fold3/kobert/0f_9e_0.8895_s.pth")
    _, logit2 = run_predict("./saved_models/fold3/kobert/1f_10e_0.8823_s.pth")
    _, logit3 = run_predict("./saved_models/fold3/kobert/2f_8e_0.8888_s.pth")
    _, logit4 = run_predict("./saved_models/fold3/kobert/3f_10e_0.8897_s.pth")
    _, logit5 = run_predict("./saved_models/fold3/kobert/4f_8e_0.8867_s.pth")
    final_logit += (logit1+logit2+logit3+logit4+logit5)/5
    #여기선 레이블은 안 쓰고 logit만 사용
    #예측 로짓의 평균을 취해 final_logit에 추가함
    #####################
    return final_logit

In [ ]:
final_logit = ensemble()

In [ ]:
sub = pd.read_csv("./sample_submission.csv")
sub['topic_idx'] = final_logit.argmax(1)
# preds
sub.to_csv('./submission/final_submission.csv', index=False)